## Weak 36

In [ ]:
import tiktoken
from tokenizers import Tokenizer
from datasets import load_dataset
import polars as pl

In [ ]:
# Get huggingface dataset
dataset = load_dataset("coastalcph/tydi_xor_rc")
df_train = pl.from_pandas(dataset["train"].to_pandas())
df_test = pl.from_pandas(dataset["test"].to_pandas())

In [ ]:
df_train.columns

In [ ]:
df_train.head()

In [ ]:
df_ar = df_train.filter(pl.col("lang") == "ar")
df_ko = df_train.filter(pl.col("lang") == "ko")
df_te = df_train.filter(pl.col("lang") == "te")

In [ ]:
df_te.describe()

In [ ]:
# Load multilingual BERT tokenizer
mbert_tokenizer = Tokenizer.from_pretrained("bert-base-multilingual-uncased")
# Load GPT-4 tokenizer
gpt4_tokenizer = tiktoken.get_encoding("cl100k_base")

In [ ]:
" | ".join(mbert_tokenizer.encode(df_ar["question"][0]).tokens)

In [ ]:
# Decode each token from GPT-4 tokenizer
" | ".join([gpt4_tokenizer.decode([token]) for token in gpt4_tokenizer.encode(df_ar["question"][0])])


## Get the top 5 most frequent words

In [ ]:
df_ar.with_columns(
    pl.col("question")
    .map_elements(lambda x: mbert_tokenizer.encode(x).tokens)
    .alias("tokens")
)